#### Capstone Project
Arthur: Wilson Lau
Date: 2015 November

### Capstone Milestone Report

## Introduction
The problem I want to tackle in my capstone project is to detect investment opportunities for cash flow real estate property.  

## The Problem
Real estate investment is a passive income opportunity for investors to accumulate return without active management.  Huge long term appreciation is very much possible for real estate.  However, appreciation often happens in short period of time and the appreciation is heavily affected local market condition.  It is difficult for out-of-the-market investors to detect the trend and acquire properties before price goes up.  My capstone project is trying to help investors to detect real estate investment opportunities and be able to predict which zip code will experience long term appreciation in the near future. 

My project is divided into two stages:

# Stage One
- the first stage is to use anomaly detection technique to discover which zipcodes have success sprike of upward price appreciation. 
- The Anomaly detection is compared new monthly updated home price with historical price as well as comparing with the home price of the surrending areas based on neighborhoods, city, menpolian area, and state.

# Stage Two
- Based on the result of stage one, more data will be collected for the zipcodes that have sudden increase of home price.  
- Those new dataset will be used to figure out what feature contributes to the price appreciation and a predictive model will be created using those features. 

## Dateset
Dataset used is downloaded from Quandl's Zillow data.  This dataset is a monthly time series recording the change of home price. 

# What important fields and information does the data set have?
The important columns for this dataset is time(date end of the month), home price and zipcode. 



# What are its limitations i.e. what are some questions that you cannot answer with this data set?
This dataset actually have 22 columns.  For example, it has columns for different home size e.g. 1 bedroom, 2 bedroom, 3 bedroom, price per square foot, average listing price, etc...  Since it seems that most of these columns are highly corelate to home price, they may be very help for predicting home price. 

# What kind of cleaning and wrangling did you need to do?
Quandl's API is very powerful and very flexible.  At the same time, it is a bit complicated to pull the home price.  One way to pull home price is to pass zipcode as input parameter, and Quandl will return the historical price data with all the columns related to that zipcode.  So, to get the data, I will need to use a zipcode dataset that contains all the zipcode of the US and then use the Quandl API to pull data one zipcode at a time. 

# Are there other datasets you can find, use and combine with, to answer the questions that matter?
To order to group home price data by city, state, etc.., I will need to find other dataset that relates individual zipcodes with their city name and state name.  I will need to merge the dataset together in order to find out which zipcode belongs to which city or state. 

## Preliminary Findings and Hypothesis test.
.....






In [126]:
''' Problem: Try to find which zipcode and in which months has its price increase sigificantly higher than the mean price with
the same zipcode and also than the rest of the zipcode. '''

#load home price data into dataframe
import numpy as np
import pandas as pd
import pickle
from scipy import stats

#load home price data from Datafiles
df = pickle.load( open( ".\DataPfiles\93063_to_93524.p", "rb" ) ) 

###pivot All Homes prirce to row and grouped them by zipcode
df = df.reset_index()
dfAllHomes = df.pivot(index='index', columns='ZipCode', values='All Homes')

### calcaluate the home price's moving average for every 3 months, with minimal of 3 months
dfAllHomesMovingAve = pd.rolling_mean(dfAllHomes,window=3,min_periods=3)

### shift entire set of Moving Average number downward for one month
### This is needed to line up the previous month moving average with the current month moving average in the same row
dfAllHomesMovingAve = dfAllHomesMovingAve.shift(periods=1,freq=None,axis=0)

### since I shift the all rows for one month earlier, so I can just subtract the two dataframe and calculate the price change.
dfAllHomesDiffFromMovAve = dfAllHomes - dfAllHomesMovingAve
dfAllHomesStdDevInZipCode = dfAllHomesDiffFromMovAve.copy() #dataframe to hold z score within zipcode

##loop through each zipcode column
for x in dfAllHomesStdDevInZipCode.columns:
    if sum(dfAllHomesStdDevInZipCode[x].isnull()) > 0:  # if any month has null value, drop the entire zipcode
        dfAllHomesStdDevInZipCode.drop(x,axis=1)
    dfAllHomesStdDevInZipCode[x] = (dfAllHomesStdDevInZipCode[x] - dfAllHomesStdDevInZipCode[x].mean()) / dfAllHomesStdDevInZipCode[x].std()  #calculate z score for All Homes price each month

##Calculate z score comparing price across zipcode
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevInZipCode.copy() #dataframe to hold z score across different zipcode
dfAllHomesStdDevAcrossZipCode['MeanHomePrice'] = dfAllHomesStdDevAcrossZipCode.mean(axis=1)
dfAllHomesStdDevAcrossZipCode['StdDevHomePrice'] = dfAllHomesStdDevAcrossZipCode.std(axis=1)
dfAllHomesStdDevAcrossZipCode = (dfAllHomesStdDevAcrossZipCode - dfAllHomesStdDevAcrossZipCode['MeanHomePrice']) / dfAllHomesStdDevAcrossZipCode['StdDevHomePrice']

##drop the two added columns so that match the dataframe size of df..InZipCode and df..AcrossZipCode
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('MeanHomePrice',axis=1)
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('StdDevHomePrice',axis=1)

##Find out which zipcode and its time frame in the dataframe has z score large than 2 in both df..InZipCode and df..AcrossZipCode dataframe
targetZscore = 2
targetZipcodeBoolean = ((dfAllHomesStdDevInZipCode > targetZscore) & (dfAllHomesStdDevAcrossZipCode > targetZscore))
targetZipCodes = dfAllHomesMovingAve[targetZipcodeBoolean].dropna
targetZipCodes

# dfAllHomesMovingAve[targetZipcodeBoolean].sum(axis = 0)

<bound method DataFrame.dropna of ZipCode     93063  93065          93066  93067  93101          93103   93105  \
index                                                                          
1996-04-30    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-05-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-06-30    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-07-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-08-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-09-30    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-10-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-11-30    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1996-12-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1997-01-31    NaN    NaN            NaN    NaN    NaN            NaN     NaN   
1997-0

In [89]:
### This cell is trying to calculate the distance between two zipcodes (within California) in miles
import pandas as pd
import pickle
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from math import radians, cos, sin, asin, sqrt
def calculateDistance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km


geolocator = Nominatim()

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

startzipcodeIndex = int(100)    #last batch processed by script at test.ipynb
endzipcodeIndex = int(200)

for z in calzipcode.iloc[startzipcodeIndex:endzipcodeIndex,0]:
# for z in calzipcode.iloc[2:3,0]:
# for z in calzipcode['Postal Code']:    
    for i,r in calzipcode.iterrows():
        ziplon1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Longitude']
        ziplat1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Latitude']
        ziplon2 = r['Longitude']
        ziplat2 = r['Latitude']
        calzipcode.loc[i,z] = calculateDistance(ziplon1,ziplat1,ziplon2,ziplat2)

dfname = './DataPfiles/' + 'calzip' + str(int(calzipcode.iloc[startzipcodeIndex]['Postal Code'])) + "_to_" + str(int(calzipcode.iloc[endzipcodeIndex-1]['Postal Code'])) + ".p"
pickle.dump( calzipcode, open( dfname, "wb" ) )
print 'Done: ' + dfname
 
    
# geolocator = Nominatim()
# location = geolocator.geocode("94582")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)

# # >> newport_ri = (41.49008, -71.312796)
# # >>> cleveland_oh = (41.499498, -81.695391)
# print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)



Done: ./DataPfiles/calzip90185_to_90620.p


In [1]:
### This cell as well as the cell after this one are just my playground to play with different functions.  This is not part of the project. 


from pylab import plot, ylim, xlim, show, xlabel, ylabel
from numpy import linspace, loadtxt
import numpy as np

r=3.0

x = p.head()
y = pz

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

# plot(x,y)
# xlim(0,1000)

x_av = movingaverage(x, r)
# plot(x_av, y)

# xlabel("Months since Jan 1749.")
# ylabel("No. of Sun spots")
# show()
print x_av

p = df11.iloc[:,0]  #ALl Homes price
pz = df11.iloc[:,-1] 
print pz


t1 = df.iloc[:,1] 
t2 = df3.iloc[:,0]
print "mean is % 4.3F and sd is % 4.3F " % (t1.mean(),t1.std
                                            
                                            # add a new column for moving average of All Homes price


p = df11.iloc[:,0] #ALl Homes price
pz = df11.iloc[:,-1] 


window_size = 3.0 #set the number of sample to gathering centered in the middle
movingave = lambda x: np.convolve(x, np.ones(int(window_size))/float(window_size), 'same')
transformed = df11.groupby('ZipCode')
transformed['All Homes'].transform(movingave)
# df11.info()

# df11 = df11[df11['All Homes'].isnull()]
grouped = df11[['All Homes','ZipCode']].groupby('ZipCode')
b = pd.DataFrame()
newdf = pd.DataFrame()
for name,group in grouped:
    g = group.copy()
#     print group.shape
#     print "size: % 3.2F" % movingaverage(group['All Homes'],3).size
#     if sum(g.isnull()) < 0:
#         g['Moving Ave'] = movingaverage(group['All Homes'],3)

#     print g.shape
#     print g.head()

    #     if(newdf.isnull):
#         newdf = g
#     newdf = newdf.append(g)
#     b = group['All Homes']
#     a = movingaverage(group['All Homes'],3)
    
#     group["MovingAve"] = np.convolve(group['All Homes'], np.ones(int(window_size))/float(window_size), 'same')
#     newdf = newdf.append(group)
# print b
# newdf
# newdf.info()
    
    


# df11['Moving Ave'] = movingaverage(df11.iloc[:,0], 3)  #para#1 is All Homes price; para#2 is numer of sample to gather centered in the middle

# df11.head()
x                                             

SyntaxError: invalid syntax (<ipython-input-1-2f2d3327462f>, line 40)

In [57]:
np.sum(dfAllHomesStdDevAcrossZipCode > 2)


ZipCode
93063               0
93065               1
93066              16
93067               0
93101               3
93103               6
93105               3
93108              20
93109              10
93110               7
93111               2
93117               7
93201               0
93202               7
93203               0
93204              11
93205               0
93206               0
93207               0
93208               0
93210               0
93212               0
93215               3
93218               0
93219              16
93221               4
93222              10
93223               2
93224               0
93225               4
                   ..
93442               2
93444               1
93445               5
93446               5
93449              17
93450               0
93451              14
93452               0
93453              11
93454               0
93455               1
93458               2
93460              12
93461               0
93

In [21]:
np.average(dfAllHomes['93063'][(-1-y):-1])

483400.0

In [12]:
import pandas as pd
dfAllHomesDiffFromMovAve

ZipCode,93063,93065,93066,93067,93101,93103,93105,93108,93109,93110,...,93505,93510,93512,93513,93514,93516,93517,93518,93519,93523
index,,,,,,,,,,,,,,,,,,,,,
1996-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-07-31,100.000000,-200.000000,-11100.000000,NaN,-1766.666667,-8666.666667,3466.666667,NaN,-66.666667,-6100.000000,...,-533.333333,-8866.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-08-31,1000.000000,33.333333,-2433.333333,NaN,-133.333333,-4200.000000,6100.000000,NaN,800.000000,-3266.666667,...,-200.000000,-5833.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-09-30,733.333333,566.666667,3866.666667,NaN,1800.000000,2400.000000,6666.666667,NaN,1466.666667,400.000000,...,-766.666667,-2666.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-10-31,-200.000000,600.000000,2000.000000,NaN,3466.666667,7200.000000,6233.333333,NaN,1800.000000,1533.333333,...,-1000.000000,-533.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-11-30,-600.000000,300.000000,-1166.666667,NaN,3466.666667,7666.666667,4966.666667,NaN,-166.666667,-1066.666667,...,-866.666667,833.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-12-31,-566.666667,466.666667,-66.666667,NaN,1900.000000,4200.000000,2700.000000,NaN,-2133.333333,-5300.000000,...,-566.666667,1466.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
print dfAllHomesStdDevAcrossZipCode.shape
dfAllHomesStdDevAcrossZipCode
# np.sum(dfAllHomesStdDevAcrossZipCode > 2)

(231, 127)


ZipCode,93063,93065,93066,93067,93101,93103,93105,93108,93109,93110,...,93512,93513,93514,93516,93517,93518,93519,93523,MeanHomePrice,StdDevHomePrice
index,,,,,,,,,,,,,,,,,,,,,
1996-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-07-31,0.002422,-0.084785,-3.253319,NaN,-0.540201,-2.545970,0.981083,NaN,-0.046026,-1.799863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.973353
1996-08-31,0.141966,-0.189977,-1.037005,NaN,-0.247209,-1.643660,1.893254,NaN,0.073288,-1.323163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.798576
1996-09-30,-0.072360,-0.144320,1.280493,NaN,0.388186,0.647242,2.489425,NaN,0.244265,-0.216281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.611015
1996-10-31,-0.550377,-0.107203,0.668352,NaN,1.480838,3.548984,3.013482,NaN,0.557558,0.409834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.560416
1996-11-30,-0.657150,-0.108867,-1.002365,NaN,1.820276,4.378930,2.734081,NaN,-0.393162,-0.941444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.708372
1996-12-31,-0.485677,0.235568,-0.136688,NaN,1.236004,2.841356,1.794387,NaN,-1.579177,-3.789444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.909844


In [51]:
dfAllHomesDiffFromMovAve.mean(axis =1 )

index
1996-04-30             NaN
1996-05-31             NaN
1996-06-30             NaN
1996-07-31       91.666667
1996-08-31      586.574074
1996-09-30      900.925926
1996-10-31      793.518519
1996-11-30      478.703704
1996-12-31      129.166667
1997-01-31       85.648148
1997-02-28      405.555556
1997-03-31      780.092593
1997-04-30      760.648148
1997-05-31      875.877193
1997-06-30      813.157895
1997-07-31      971.491228
1997-08-31      860.526316
1997-09-30      707.456140
1997-10-31      701.315789
1997-11-30      964.935065
1997-12-31     1424.675325
1998-01-31     2119.480519
1998-02-28     2412.554113
1998-03-31     1835.064935
1998-04-30     2538.961039
1998-05-31     1176.666667
1998-06-30     1123.333333
1998-07-31     1037.777778
1998-08-31      933.333333
1998-09-30     1007.777778
                  ...     
2013-01-31     6131.034483
2013-02-28     6220.689655
2013-03-31     6819.540230
2013-04-30     7923.371648
2013-05-31     9233.333333
2013-06-30    10330.26

In [1]:
dfAllHomesDiffFromMovAve.std(axis = 1)

NameError: name 'dfAllHomesDiffFromMovAve' is not defined

In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

geolocator = Nominatim()
location = geolocator.geocode("94582")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

# >> newport_ri = (41.49008, -71.312796)
# >>> cleveland_oh = (41.499498, -81.695391)
print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)

San Ramon, Contra Costa County, California, 94582, United States of America
(37.7608662693782, -121.903546728611)
{u'display_name': u'San Ramon, Contra Costa County, California, 94582, United States of America', u'importance': 0.225, u'place_id': u'128955624', u'lon': u'-121.903546728611', u'lat': u'37.7608662693782', u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', u'boundingbox': [u'37.760816269378', u'37.760916269378', u'-121.90359672861', u'-121.90349672861'], u'type': u'postcode', u'class': u'place'}
31.8138952547


In [46]:
import pandas as pd
import pickle

df = pickle.load( open( ".\DataPfiles\calzip90001_to_90010.p", "rb" ) ) 

# print "abcde"
# zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
# calzipcode = zipcode[zipcode.State == 'California']
df

,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7,90001,90002,90003,90004,90005,90006,90007,90008,90009,90010
39116,90001,Los Angeles,California,CA,Los Angeles,33.9731,-118.2479,NaN,0.000000,2.605044,2.444413,12.526859,10.684774,9.379803,7.223813,9.593777,21.248139,10.955510
39117,90002,Los Angeles,California,CA,Los Angeles,33.9497,-118.2462,NaN,2.605044,0.000000,2.995232,14.996201,13.109151,11.835250,9.625065,11.125998,18.760691,13.377814
39118,90003,Los Angeles,California,CA,Los Angeles,33.9653,-118.2727,NaN,2.444413,2.995232,0.000000,12.633782,10.684404,9.497183,7.245611,8.135811,20.002416,10.944822
39119,90004,Los Angeles,California,CA,Los Angeles,34.0762,-118.3029,NaN,12.526859,14.996201,12.633782,0.000000,1.973130,3.162086,5.400275,7.994109,32.184163,1.733650
39120,90005,Los Angeles,California,CA,Los Angeles,34.0585,-118.3012,NaN,10.684774,13.109151,10.684404,1.973130,0.000000,1.345442,3.484639,6.376781,30.215798,0.271160
39121,90006,Los Angeles,California,CA,Los Angeles,34.0493,-118.2917,NaN,9.379803,11.835250,9.497183,3.162086,1.345442,0.000000,2.251593,6.184544,29.199703,1.613214
39122,90007,Los Angeles,California,CA,Los Angeles,34.0294,-118.2871,NaN,7.223813,9.625065,7.245611,5.400275,3.484639,2.251593,0.000000,5.352551,27.002379,3.752879
39123,90008,Los Angeles,California,CA,Los Angeles,34.0116,-118.3411,NaN,9.593777,11.125998,8.135811,7.994109,6.376781,6.184544,5.352551,0.000000,25.307152,6.492606
39124,90009,Los Angeles,California,CA,Los Angeles,33.7866,-118.2987,NaN,21.248139,18.760691,20.002416,32.184163,30.215798,29.199703,27.002379,25.307152,0.000000,30.450515
39125,90010,Los Angeles,California,CA,Los Angeles,34.0606,-118.3027,NaN,10.955510,13.377814,10.944822,1.733650,0.271160,1.613214,3.752879,6.492606,30.450515,0.000000


In [87]:
import os
from os import path
files = [f for f in os.listdir(".\DataPfiles\\")]# if path.isfile(f)]
files
# print os.listdir(".\DataPfiles\\")

['90621_to_91024.p',
 '93063_to_93524.p',
 'calzip90001_to_90010.p',
 'calzip90002_to_90001.p',
 'calzip90002_to_90005.p',
 'calzip90003_to_90003.p',
 'calzip90011_to_90020.p',
 'calzip90021_to_90050.p',
 'us_postal_codes.csv']